In [5]:
import torch
input = torch.randn(1, 1, 96, 96, 96)
print(input.shape)
from torchkeras import summary
from thop import profile
from monai.networks.nets import SwinUNETR
# from prostate158.network.davitUNETR_v3_2_dca import davitUNETR, ChannelBlock, SpatialBlock
import torch.nn as nn
import torch.nn.functional as F


torch.Size([1, 1, 96, 96, 96])


In [2]:
Y1 = torch.randn(343,343,48)
Y2 = torch.randn(343,343,48)

advavgpool = nn.AdaptiveAvgPool2d(1)
out = torch.cat([Y1,Y2], dim=2)
print(out.shape)

torch.Size([343, 343, 96])


In [6]:
# advavgpool(out).shape
out     = F.softmax(advavgpool(out), dim=2 ) * out

In [8]:
out1,out2 = torch.split(out,out.size(2)//2,dim=2)

(torch.Size([343, 343, 48]), torch.Size([343, 343, 48]))

In [21]:
model1 = SpatialBlock(
            dim=96,
            num_heads=3,
            window_size=(7,7,7),
            shift_size=(0,0,0), # if (i % 2 == 0) else self.shift_size, 
            mlp_ratio=4.0,
            qkv_bias=True,
            drop=0.0,
            attn_drop=0.0,
            drop_path=0.0,
            norm_layer=nn.LayerNorm,
            use_checkpoint=False,
            qkv_mode='',
                ).to('cpu')

model2 = ChannelBlock(
            dim=96,
            num_heads=3,
            mlp_ratio=4.0,
            qkv_bias=True,
            drop_path=0.0,
            norm_layer=nn.LayerNorm,
            ffn=True,
            qkv_mode='',
            ).to('cpu')

In [22]:
input = torch.randn(2, 24, 24, 24, 96)
attn_mask = torch.zeros(343,343,343)
output = model1(input, attn_mask)

In [23]:
flops, params = profile(model1, inputs=(input,attn_mask))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')
from torchkeras import summary
summary(model1, input_data_args=(input,attn_mask))

[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv3d'>.
FLOPs = 3.956869632G
Params = 0.117216M
--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv3d-1                        [-1, 96, 24, 24, 24]                2,688
LayerNorm-2                          [-1, 13824, 96]                  192
Linear-3                              [-1, 343, 288]               27,936
Softmax-4                          [-1, 3, 343, 343]                    0
Dropout-5                          [-1, 3, 343, 343]                    0
Linear-6                             

'--------------------------------------------------------------------------\nLayer (type)                            Output Shape              Param #\n==========================================================================\nConv3d-1                        [-1, 96, 24, 24, 24]                2,688\nLayerNorm-2                          [-1, 13824, 96]                  192\nLinear-3                              [-1, 343, 288]               27,936\nSoftmax-4                          [-1, 3, 343, 343]                    0\nDropout-5                          [-1, 3, 343, 343]                    0\nLinear-6                               [-1, 343, 96]                9,312\nDropout-7                              [-1, 343, 96]                    0\nIdentity-8                           [-1, 13824, 96]                    0\nConv3d-9                        [-1, 96, 24, 24, 24]                2,688\nIdentity-10                     [-1, 24, 24, 24, 96]                    0\nLayerNorm-11          

In [24]:
flops, params = profile(model2, inputs=(input,attn_mask))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')
from torchkeras import summary
summary(model2, input_data_args=(input,attn_mask))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv3d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
FLOPs = 3.222208512G
Params = 0.117216M
--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv3d-1                        [-1, 96, 24, 24, 24]                2,688
LayerNorm-2                          [-1, 13824, 96]                  192
Linear-3                            [-1, 13824, 288]               27,936
Linear-4                             [-1, 13824, 96]                9,312
Identity-5                           [-1, 13824, 96]                    0
Conv3d-6                        [-1, 96, 24, 24, 24]                2,688
LayerNorm-7                          [-1, 1382

'--------------------------------------------------------------------------\nLayer (type)                            Output Shape              Param #\n==========================================================================\nConv3d-1                        [-1, 96, 24, 24, 24]                2,688\nLayerNorm-2                          [-1, 13824, 96]                  192\nLinear-3                            [-1, 13824, 288]               27,936\nLinear-4                             [-1, 13824, 96]                9,312\nIdentity-5                           [-1, 13824, 96]                    0\nConv3d-6                        [-1, 96, 24, 24, 24]                2,688\nLayerNorm-7                          [-1, 13824, 96]                  192\nLinear-8                            [-1, 13824, 384]               37,248\nGELU-9                              [-1, 13824, 384]                    0\nDropout-10                          [-1, 13824, 384]                    0\nLinear-11             